In [1]:
!pip install monai nibabel torchvision pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.5 MB/s eta 0:00:00


In [26]:
!pip install pytorch_lightning

In [115]:
import os
import nibabel as nib
import torch
import numpy as np
import pydicom

import torch
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as L

from monai.transforms import Compose, RandRotate90d, RandFlipd, RandZoomd

In [116]:
dicom_dir = '/content/drive/MyDrive/FLAIR'
nifti_output_path = '/content/drive/MyDrive/NIFTI/output.nii.gz'

In [126]:
import os

directory = '/content/drive/MyDrive/NIFTI'
files = os.listdir(directory)
print(f"Files in {directory}:")
for file in files:
    print(file)

Files in /content/drive/MyDrive/NIFTI:
output.nii.gz


In [127]:
import os

dicom_dir = '/content/drive/MyDrive/FLAIR'
if os.path.exists(dicom_dir):
    print(f"The DICOM directory exists at {dicom_dir}")
    print("Files in the directory:")
    for file in os.listdir(dicom_dir):
        print(file)



else:
    print(f"The DICOM directory does not exist at {dicom_dir}")

The DICOM directory exists at /content/drive/MyDrive/FLAIR
Files in the directory:
BRAINIX_DICOM_FLAIR_IM-0001-0011.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0019.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0004.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0008.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0007.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0017.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0003.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0013.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0015.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0018.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0006.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0016.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0002.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0020.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0021.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0009.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0005.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0001.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0012.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0014.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0022.dcm
BRAINIX_DICOM_FLAIR_IM-0001-0010.dcm
.DS_Store


In [129]:
# Conversion from dicom to nifti


import os
import numpy as np
import pydicom
import nibabel as nib

def dicom_to_nifti(dicom_dir, nifti_output_path):
    slices = []  # will store the pixel data from dicom files

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(nifti_output_path), exist_ok=True)

    for s in sorted(os.listdir(dicom_dir)):  # Sort to ensure correct slice order
        if s.endswith('.dcm'):
            dcm_path = os.path.join(dicom_dir, s)
            dcm = pydicom.dcmread(dcm_path)
            slices.append(dcm.pixel_array)

    if not slices:
        raise ValueError(f"No DICOM files found in {dicom_dir}")

    volume = np.stack(slices, axis=-1)  # created 3d volume by stacking the np arrays
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))  # NIFTI object conversion
    nib.save(nifti_img, nifti_output_path)  # saving
    print(f"NIFTI file saved at {nifti_output_path}")

dicom_dir = '/content/drive/MyDrive/FLAIR'


nifti_output_path = '/content/drive/MyDrive/NIFTI/output.nii.gz' # NIFTI Files here lessgoo


# Run the conversion
try:
    dicom_to_nifti(dicom_dir, nifti_output_path)
except Exception as e:
    print(f"An error occurred during conversion: {str(e)}")

NIFTI file saved at /content/drive/MyDrive/NIFTI/output.nii.gz


In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
def normalise_volume(volume):
    volume = (volume - np.mean(volume))/np.std(volume) # this is normalising along std and mean

    return volume

In [119]:
from monai.transforms import Resize

## Dataset class

In [120]:
# creating dataset class

class MRI_Dataset(Dataset):
    def __init__(self, nifti_files, transform=None, target_shape=(288, 288, 32)):
        self.nifti_files = nifti_files
        self.transform = transform
        self.target_shape = target_shape # ensure that the target shape is 32 even when the 288 * 288 * 22 is padded here
        self.resize = Resize(spatial_size=target_shape)


    def __len__(self):
        return len(self.nifti_files) # return the lenght



    def __getitem__(self, idx):
        nifti_file = self.nifti_files[idx]
        img = nib.load(nifti_file).get_fdata()
        img = normalise_volume(img)
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        # Pad the third dimension from 22 to 32
        pad_size = self.target_shape[2] - img.shape[3]
        img = torch.nn.functional.pad(img, (0, pad_size, 0, 0, 0, 0)) # padding

        if self.transform:
            img = self.transform(img)

        return img, img  # input and target are the same for autoencoders

## Augmentation

In [121]:
# performing Data augmentation and transofmration

import torch
from monai.transforms import (
    Compose,
    RandRotate90,
    RandFlip,
    RandZoom,
    EnsureType
)

transform = Compose([
    EnsureType(),
    RandRotate90(prob=0.5, spatial_axes=[0, 1]),
    RandFlip(prob=0.5, spatial_axis=0),
    RandZoom(prob=0.5, min_zoom=0.9, max_zoom=1.1),
])

dataset = MRI_Dataset(nifti_files=nifti_files, transform=transform, target_shape=(288, 288, 32))
nifti_files = ['/content/drive/MyDrive/NIFTI/output.nii.gz']  # final nifti destination

In [122]:
def check_dataset(dataset):
    print(f"Dataset length: {len(dataset)}")
    sample, _ = dataset[0]
    print(f"Sample shape: {sample.shape}")
    print(f"Sample dtype: {sample.dtype}")
    print(f"Sample min: {sample.min()}, max: {sample.max()}")

check_dataset(dataset)

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # Batch size 1 for testing


Dataset length: 1
Sample shape: torch.Size([1, 288, 288, 32])
Sample dtype: torch.float32
Sample min: -0.6572733521461487, max: 7.510774612426758


### Dataloader creation

In [123]:
# Create dataset and dataloader
dataset = MRI_Dataset(nifti_files=nifti_files, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [124]:
from monai.networks.nets import SwinUNETR

model = SwinUNETR(
    img_size=(288, 288, 32),  # Adjusted to match our padded input
    in_channels=1,
    out_channels=1,
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,

    dropout_path_rate=0.0,
    use_checkpoint=False, # no need for small dataset
)



# Imports swin unter and initialises the model
import torch.optim as optim

In [128]:
import torch.nn as nn
#loss fuinction

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4) # adam optimiser with lr


# checks if cuda availane

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): Linear(in_feature

Input shape: torch.Size([1, 1, 288, 288, 32])


In [130]:
# verifying is file exists

import os

nifti_file_path = '/content/drive/MyDrive/NIFTI/output.nii.gz'
if os.path.exists(nifti_file_path):
    print(f"The file exists at {nifti_file_path}")
else:
    print(f"The file does not exist at {nifti_file_path}")

The file exists at /content/drive/MyDrive/NIFTI/output.nii.gz


In [131]:
# some image checking -

img = nib.load(nifti_file_path).get_fdata()
print(img.shape)

(288, 288, 22)


In [1]:
num_epochs = 50

In [2]:
# Test a single batch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_input, _ = next(iter(dataloader))
test_input = test_input.to(device)
print(f"Input shape: {test_input.shape}")
try:
    test_output = model(test_input)
    print(f"Output shape: {test_output.shape}")
    print("Model successfully processed the input")
except Exception as e:
    print(f"Error occurred: {str(e)}")


    # the testing failed as the MRI images are too large for my ram to process at this point
    # also worked with pytorch lightning and created custom data module and lightning module to fit the model


NameError: name 'torch' is not defined

In [ ]:
# nasic traiing loop here


for epoch in range(num_epochs):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)


        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Batch idx: {batch_idx}, Loss: {loss.item()}")